In [ ]:
from ipyradiant import (
    CustomURIRef,
    FileManager,
    MultiPanelSelect,
    ObjectLiteralApp,
    PathLoader,
    collapse_preds,
)
from rdflib import URIRef

## Start by using the `ObjectLiteralApp`

### We also utilize the `CustomURIRef` class from `ipyradiant`. This allows us to get a 'pretty' representation of a URIRef while mainting access to the valuable URIRef.

We can borrow the loading widgets seen in other examples like the Tab App in order to
get a graph to play around with.

In [ ]:
lw = FileManager(loader=PathLoader(path="data"))
lw

Now we can use the `ObjectLiteralApp` to move predicates around based on what we which
ones we want to select. The 'Add predicates where object is a literal' button will
automatically move all predicates to the 'predicates to collapse' side if that predicate
is _always_ associated with a literal object.

In [ ]:
graph = lw.graph
ola = ObjectLiteralApp(graph=graph, namespaces={URIRef("http://www.w3.org/"): "w3"})
ola

Now lets say we want to access one of the predicates. We can do this while retaining
full access to the URI as follows:

In [ ]:
try:
    first_pred = ola.multiselect.selected_things_list[0]
    print(first_pred.uri)
except:
    print("need to put some data in the graph and move it around for this to work.")

## Transformations of Graph

In [ ]:
import networkx as nx
import rdflib
from rdflib.extras.external_graph_libs import (
    rdflib_to_networkx_graph,
    rdflib_to_networkx_multidigraph,
)

In [ ]:
netx_graph = rdflib_to_networkx_graph(graph)

We will collapse all the predicates that consistently have literal objects, and then see
how it effects the graph.

In [ ]:
preds_to_collapse = ola.multiselect.selected_things_list
preds_to_collapse = [_.uri for _ in preds_to_collapse]

Here we get a list of all subject nodes so that we do not remove an object that is also
a subject.

In [ ]:
subjects = [s for s, o in netx_graph.edges]

Now we can collapse down the data in the graph, and then later take a look at how that
impacted the visualization of the graph.

In [ ]:
# loop through edges, if the predicate is in this list, then flag to remove
netx_graph = collapse_preds(
    netx_graph,
    preds_to_collapse,
    subjects,
    namespaces={URIRef("http://www.w3.org/"): "w3"},
)

In [ ]:
try:
    print(netx_graph.nodes[list(netx_graph.nodes)[0]])
except:
    print("need to have a non-empty networkx graph")